<a href="https://colab.research.google.com/github/alicececot/ProgettoML-SI/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/ProgettoML/tweet_emotions.csv')
df.head()
SEED = 19 # Supponendo che il tuo DataFrame si chiami df
df_sampled = df.sample(frac=0.5, random_state=SEED)  # Utilizza il 50% del dataset

# Verifica la dimensione del nuovo DataFrame
print(f"Dimensione del dataset campionato: {df_sampled.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dimensione del dataset campionato: (20000, 3)


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef

from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
     torch.cuda.manual_seed_all(SEED)

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: Dividi il dataset in Train + Validation set e Test set
train_val, test = train_test_split(df_sampled, test_size=0.1, random_state=42)  # 20% per il Test set

# Step 2: Dividi Train + Validation in Train set e Validation set
train, validation = train_test_split(train_val, test_size=0.1111, random_state=42)  # 20% del rimanente per Validation set

In [ ]:
print(f'Train set size: {len(train)}')
print(f'Validation set size: {len(validation)}')
print(f'Test set size: {len(test)}')

Train set size: 16000
Validation set size: 2000
Test set size: 2000


In [ ]:
df_sampled['sentiment'].unique()

array(['love', 'empty', 'worry', 'sadness', 'happiness', 'neutral',
       'boredom', 'fun', 'relief', 'surprise', 'enthusiasm', 'hate',
       'anger'], dtype=object)

In [ ]:
#unire tutte le emozioni che hanno meno di 1000 tweet in un'unica denominata others
# 1. Conta il numero di tweet per ciascuna emozione
sentiment_counts = df_sampled['sentiment'].value_counts()

# 2. Identifica le emozioni con meno di 1000 tweet
low_frequency_emotions = sentiment_counts[sentiment_counts < 1000].index

# 3. Sostituisci queste emozioni con l'etichetta 'others'
df_sampled['sentiment'] = df_sampled['sentiment'].apply(lambda x: 'others' if x in low_frequency_emotions else x)

# 4. Verifica se il cambio è avvenuto correttamente
print(df_sampled['sentiment'].value_counts())

sentiment
neutral      4297
worry        4221
others       3260
sadness      2620
happiness    2592
love         1896
surprise     1114
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_sampled['sentiment_enc'] = labelencoder.fit_transform(df_sampled['sentiment'])

In [ ]:
df_sampled[['sentiment','sentiment_enc']].drop_duplicates(keep='first')

,sentiment,sentiment_enc
26350,love,1
15649,others,3
27330,worry,6
1580,sadness,4
26345,happiness,0
18610,neutral,2
37895,surprise,5


In [ ]:
df_sampled.rename(columns={'sentiment':'sentiment_desc'},inplace=True)
df_sampled.rename(columns={'sentiment_enc':'sentiment'},inplace=True)

In [ ]:
## create label and sentence list
sentences = df_sampled.content.values

#check distribution of data based on labels
print("Distribution of data based on sentiment: ",df_sampled.sentiment.value_counts())



MAX_LEN =256

## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True, max_length=MAX_LEN, truncation=True, pad_to_max_length=True) for sent in sentences]
sentiments = df_sampled.sentiment.values

print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

Distribution of data based on sentiment:  sentiment
2    4297
6    4221
3    3260
4    2620
0    2592
1    1896
5    1114
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: Futur

Actual sentence before tokenization:  @ianweiqiang Interesting Combination  Have a great one ;)
Encoded Input from dataset:  [101, 1030, 4775, 19845, 14702, 5654, 5875, 5257, 2031, 1037, 2307, 2028, 1025, 1007, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
## Create attention mask
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(input_ids,sentiments,random_state=41,test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(attention_masks,input_ids,random_state=41,test_size=0.1)

In [ ]:
# convert all our data into torch tensors, required data type for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
#impostazione della dimensione del batch; durante l'addestramento i dati vengono elaborati a gruppi di queste dimensioni
batch_size = 32

#questo passaggio crea un dataset PyTorch utilizzando i tensori di input, le maschere di attenzione e le etichette.
#TensorDataset permette di raggruppare insieme questi dati per facilitarne l'uso durante l'addestramento
train_data = TensorDataset(train_inputs, train_masks, train_labels)

#Qui viene creato un campionatore casuale.
#Questo campionatore assicura che i dati vengano selezionati in modo casuale, migliorando la generalizzazione del modello.
train_sampler = RandomSampler(train_data)

#creato un DataLoader che gestisce il dataset in modo efficiente. Utilizza il campionatore casuale e divide i dati in batch della dimensione specificata.
# Il DataLoader consente di caricare i dati in memoria solo quando necessario, risparmiando risorse durante l'addestramento.
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

#passaggi uguali per il validation
validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [ ]:
#vediamo cosa c'è nel train_data
train_data[0]

(tensor([  101,  1030,  2061, 27417,  2692,  2620,  1045,  2113,  2017,  2180,
          1005,  1056,  2562,  1037,  3748,  3899,  1010,  2030,  2842,  2017,
          2180,  1005,  1056,  2022,  2583,  1999,  1996,  3663,  2000,  2224,
          2158, 13626, 11444,  1999,  2115,  2166,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
type(train_dataloader)

torch.utils.data.dataloader.DataLoader

In [ ]:
#creazione modello per 10 etichette
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=10).to(device)

# Parameters:
lr = 2e-5 #tasso di apprendimento
adam_epsilon = 1e-8 #parametro per l'ottimizzatore Adam

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

num_warmup_steps = 0

#Calcola il numero totale di passi di addestramento (aggiornamenti dei pesi) moltiplicando
#  il numero di batch nel DataLoader di addestramento per il numero di epoche.
num_training_steps = len(train_dataloader)*epochs

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm.notebook import trange

## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

for _ in trange(1,epochs+1,desc='Epoch'):
  print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
  # Calculate total loss for this epoch
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()


    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    loss = outputs[0]

    # Backward pass
    loss.backward()

    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # Update parameters and take a step using the computed gradient
    optimizer.step()

    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()

    # Update tracking variables
    batch_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = batch_loss / len(train_dataloader)
    #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])

  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables
  eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0
  # Evaluate data for one epoch
  for batch in validation_dataloader:

    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Move logits and labels to CPU
    # Move logits and labels to CPU
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    df_metrics=pd.DataFrame({'Epoch':epochs,'Actual_class':labels_flat,'Predicted_class':pred_flat})

    tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
    tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)

    eval_accuracy += tmp_eval_accuracy
    eval_mcc_accuracy += tmp_eval_mcc_accuracy #MCC particolarmente adatto per dataset sbilanciati
    nb_eval_steps += 1

  print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')
  print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy/nb_eval_steps}')



Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>

	Current Learning rate:  1.3333333333333333e-05

	Average Training loss: 1.6748879925607576

	Validation Accuracy: 0.3898809523809524

	Validation MCC Accuracy: 0.2658267020191888
<====================== Epoch 2 ======================>

	Current Learning rate:  6.666666666666667e-06

	Average Training loss: 1.4358398584023673

	Validation Accuracy: 0.3943452380952381

	Validation MCC Accuracy: 0.2675588431728394
<====================== Epoch 3 ======================>

	Current Learning rate:  0.0

	Average Training loss: 1.2665415133083269

	Validation Accuracy: 0.3913690476190476

	Validation MCC Accuracy: 0.26606140769831993


In [ ]:
print(classification_report(df_metrics['Actual_class'].values, df_metrics['Predicted_class'].values))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.25      1.00      0.40         1
           2       0.67      0.67      0.67         3
           3       0.00      0.00      0.00         5
           4       0.00      0.00      0.00         1
           6       0.50      0.50      0.50         2

    accuracy                           0.44        16
   macro avg       0.40      0.49      0.40        16
weighted avg       0.45      0.44      0.43        16

